___
# Ciência dos Dados - PROJETO 3 - INSPER

___
## Alexandre Edington

## Bruno Cury

## Matheus Soares

___

___

## INTRODUÇÃO


Não vem de hoje o grande interesse da sociedade em vídeo games, porém, há aproximadamente dez anos atrás, a visão que tínhamos sobre o ato de jogar vídeo game e sua indústria era completamente diferente.
Em 2007 os jogos com maiores ambições comerciais projetavam um lucro multimilionário e a venda dos mesmos era feita através de discos que seriam rodados em seu console de preferência.
Hoje em dia os consoles são apenas uma parcela do vasto universo de aparelhos que rodam vídeo games, e a venda destes jogos já não se da mais por discos, sua grande maioria vem de downloads e seus lucros são projetados na faixa dos biliões de dólares. Os videogames são agora, onipresentes em nossas vidas, qualquer um com um smartphone tem acesso a uma variedade de jogos, disponíveis na plataforma de download do seu sistema operacional (iOS, Android, google play), ou até nas redes sociais como Facebook, que se tornou uma grande plataforma de jogos online. 
Com isso em mente, resolvemos analisar os dados da maior plataforma de download de vídeo games da internet, afim de quantificar o impacto dos jogos no mundo por meio da análise de suas características mais importantes. E então, a partir desta análise, predizer o sucesso (ou falta dele) na arrecadação mundial de algum jogo, através do input de algumas de suas características.

___
## MINERANDO DADOS e CARACTERÍSTICAS DO DATASET

Os dados utilizados neste experimento foram retirados na integra do site data.world, onde são disponibilizados diversos bancos de dados; o Dataset escolhido é inteiramente referente às características dos jogos disponíveis na plataforma Steam. Inicialmente, o banco de dados em questão contava com mais de 70 variáveis e uma ampla quantidade de dados, porém o mesmo precisou ser filtrado, em prol da eficiência e organização foi feita a retirada de variáveis indesejadas e a retirada de dados nulos. Escolhemos também agrupar algumas variáveis, como as entradas de gênero (Indie, Action, Adventure, Casual, Strategy, RPG, Simulation, Sports, Racing), que se tornaram uma só variável, assim como a compatibilidade com Mac, Linux e Windows se tornaram uma só entrada.

In [26]:
import pandas as pd
import os
from collections import Counter
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import math as math


In [2]:
df = pd.read_csv("games-features.csv",error_bad_lines=False,sep=";")

### BASE DE DADOS

In [3]:
dados = pd.DataFrame()
dados["NOME"] = df["ResponseName"]
dados["LANCAMENTO"] = df["ReleaseDate"]
dados["ACHIEVEMENTS"] = df["AchievementCount"]
dados["LINGUAS"] = df["SupportedLanguages"]
dados["LINGUAS_N"] = df["SupportedLanguages"]
dados["INDIE"] = df["GenreIsIndie"]
dados["ACTION"] = df["GenreIsAction"]
dados["ADVENTURE"] = df["GenreIsAdventure"]
dados["CASUAL"] = df["GenreIsCasual"]
dados["STRATEGY"] = df["GenreIsStrategy"]
dados["RPG"] = df["GenreIsRPG"]
dados["SIMULATION"] = df["GenreIsSimulation"]
dados["SPORTS"] = df["GenreIsSports"]
dados["RACING"] = df["GenreIsRacing"]
dados["NUM_PROPRIETARIOS"] = df["SteamSpyOwners"]
dados["PRECO_INICIAL"] = df["PriceInitial"]
dados["PRECO_FINAL"] = df["PriceFinal"]

In [4]:
remover=["with","full","audio","support","Traditional","Simplified"]

for i in range(len(dados)):
    tempa=dados.LINGUAS_N[i].split(" ")
    for a in remover:
        if a in tempa:
            tempa.remove(a)
    dados.LINGUAS_N[i]=len(tempa)

C:\Users\Bruno\Anaconda3\lib\site-packages\ipykernel\__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [5]:
def Plataforma(df):
    a = df.PlatformWindows
    b = df.PlatformMac
    c = df.PlatformLinux
    
    if (a == "VERDADEIRO" and b == "VERDADEIRO" and c == "VERDADEIRO"): return "Windows, Linux e Mac"
    elif (a == "VERDADEIRO" and b == "VERDADEIRO" and c == "FALSO"): return "Windows e Mac"
    elif (a == "VERDADEIRO" and b == "FALSO" and c == "FALSO"): return "Windows"
    elif (a == "FALSO" and b == "VERDADEIRO" and c == "VERDADEIRO"): return "Mac e Linux"
    elif (a == "FALSO" and b == "VERDADEIRO" and c == "FALSO"): return "Mac"
    elif (a == "VERDADEIRO" and b == "FALSO" and c == "VERDADEIRO"): return "Windows e Linux"
    elif (a == "FALSO" and b == "FALSO" and c == "VERDADEIRO"): return "Linux"
    else: return "Nenhuma Plataforma"

Input = df.apply(Plataforma,axis=1)
dados["PLATAFORMAS"] = Input


def Jogabilidade(df):
    a = df.CategoryMultiplayer
    b = df.CategorySinglePlayer
    c = df.CategoryCoop
    
    if (a == "VERDADEIRO" and b == "VERDADEIRO" and c == "VERDADEIRO"): return "MultiPlayer, SinglePlayer e Coop"
    elif (a == "VERDADEIRO" and b == "VERDADEIRO" and c == "FALSO"): return "MultiPlayer e SinglePlayer"
    elif (a == "VERDADEIRO" and b == "FALSO" and c == "FALSO"): return "MultiPlyer"
    elif (a == "FALSO" and b == "VERDADEIRO" and c == "VERDADEIRO"): return "SinglePlayer e Coop"
    elif (a == "FALSO" and b == "VERDADEIRO" and c == "FALSO"): return "SinglePlayer"
    elif (a == "VERDADEIRO" and b == "FALSO" and c == "VERDADEIRO"): return "MultiPlayer e Coop"
    elif (a == "FALSO" and b == "FALSO" and c == "VERDADEIRO"): return "Coop"
    else: return "N/"
    
Input2 = df.apply(Jogabilidade,axis=1)
dados["JOGABILIDADE"] = Input2



### ANÁLISE DESCRITIVA

___
## MODELOS DE PREDIÇÃO

[O tópico C.PROCESSO E ESTATÍSTICAS DE VALIDAÇÃO estará dentro deste tópico. Organize-o aqui de forma clara!]

[ORGANIZE AQUI TODOS OS MODELOS DE PREDIÇÃO NECESSÁRIOS NESTE PROJETO!]

o MODELO DE PREDIÇÃO PELA MÉDIA

o MODELO DOS K VIZINHOS MAIS PRÓXIMOS (K-Nearest Neighbors Regression)

o MODELO DE REGRESSÃO LINEAR (Multiple Linear Regression)

o MODELO DE ÁRVORES DE REGRESSÃO (Decision Tree Regression)

In [53]:
#MODELO DE PREDICAO PELA MEDIA

y = dados.NUM_PROPRIETARIOS.mean()
f = dados.NUM_PROPRIETARIOS.mean()
yi = dados.NUM_PROPRIETARIOS
n = len(dados)


In [70]:
# coeficiente de determinação (R2) E raiz do erro médio quadrático (RMS).
def CoefDet(y,f,yi):
    
    num = ((yi-f)**2).sum()
    den = ((yi-y)**2).sum()
    R2 = 1 - (num/den)
    
    return[R2]

In [71]:
def RSM(y,f,yi):
    som = ((yi-y)**2).sum()
    rsm = (1/n)*som
    
    return[rsm]


___
## CONCLUSÃO

___
## REFERÊNCIAS BIBLIOGRÁFICAS